In [3]:
import numpy as np
import pandas as pd

On se connecte à la base de données

In [4]:
from db_utils import get_db_engine

# Obtenir le moteur de base de données
engine = get_db_engine()

In [5]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\DietSurvey_toJson.json'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_json(file2)


In [ ]:
# food1 - df1
food1 = df1[['FCVC', 'FAVC']].rename(
    columns={
        'FCVC': 'vegetable_con',
        'FAVC': 'fat_con'
    }
)
food1['takeout'] = None  # Pas de colonne correspondante dans df1

print("food1 préparé.")

# food2 - df2
food2 = df2[
    [
        "What is your weekly food intake frequency of the following food categories: [Fresh vegetables]",
        "What is your weekly food intake frequency of the following food categories: [Oily, fried foods]"
    ]
].rename(
    columns={
        "What is your weekly food intake frequency of the following food categories: [Fresh vegetables]":"vegetable_con",
        "What is your weekly food intake frequency of the following food categories: [Oily, fried foods]":"fat_con"
    }
)

print("food2 préparé.")


In [ ]:
# Concatenation des DataFrames préparés
foods = pd.concat([food1, food2], ignore_index=True)
print("Drinks combiné.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Définir la taille de la figure pour afficher plusieurs graphes
plt.figure(figsize=(15, 15))

# Loop à travers chaque colonne du DataFrame pour créer un histogramme
for i, column in enumerate(foods.columns):
    plt.subplot(5, 3, i + 1)  # 5 lignes, 3 colonnes pour organiser les graphiques
    if foods[column].dtype == 'object':  # Si la colonne est catégorielle
        sns.countplot(data=foods, x=column)
        plt.title(f'Histogramme de {column}')
        plt.xticks(rotation=45)
    else:  # Si la colonne est numérique
        sns.histplot(foods[column], kde=True, bins=15)
        plt.title(f'Histogramme de {column}')
    
    plt.tight_layout()  # Ajuster l'espacement entre les graphiques

# Afficher tous les graphiques
plt.show()
